# 1. Import the necessary libraries for you to be able to query 4square API

In [12]:
import requests
import json
import pandas as pd
from getpass import getpass

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [13]:
import getpass

# Prompt the user, but don’t show what they type
token = getpass.getpass("Enter your API token: ")

# Now you can safely use `token` without it having appeared on-screen
print("Token length:", len(token))

Token length: 48


In [14]:
# your code here

#Ironhack: Paseo de la Chopera, 14 (Matadero - Casa del Lector, Nave Ironhack, 28045 Madrid, España.
import requests

url = "https://api.foursquare.com/v3/places/search?query=coffee&ll=40.391883%2C-3.698500"

headers = {
    "Accept": "application/json",
    "Authorization": token
}

response = requests.get(url, headers=headers)

print(response.status_code)
print(response.text)


200
{"results":[{"fsq_id":"4e7ddd972c5b81706ef2096d","categories":[{"id":13070,"name":"Argentinian Restaurant","short_name":"Argentinian","plural_name":"Argentinian Restaurants","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/argentinian_","suffix":".png"}}],"chains":[],"closed_bucket":"Unsure","distance":205,"geocodes":{"main":{"latitude":40.391515,"longitude":-3.696042}},"link":"/v3/places/4e7ddd972c5b81706ef2096d","location":{"admin_region":"Comunidad de Madrid","country":"ES","cross_street":"P. de la Chopera, 14","formatted_address":"P. de la Chopera, 14, 28045 Madrid Comunidad de Madrid","locality":"Madrid","postcode":"28045","region":"Comunidad de Madrid"},"name":"Cantina Matadero","related_places":{"parent":{"fsq_id":"4b6c7268f964a520d83a2ce3","categories":[{"id":12005,"name":"Cultural Center","short_name":"Cultural Center","plural_name":"Cultural Centers","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/building/default_","suffix":".png"}}],"name":"Matader

# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [22]:
# your code here

import requests

url = "https://api.foursquare.com/v3/places/search?query=coffee&ll=40.391883%2C-3.698500&sort=DISTANCE&limit=5"

headers = {
    "Accept": "application/json",
    "Authorization": token
}

response = requests.get(url, headers=headers)

print(response.status_code)
print(response.text)

200
{"results":[{"fsq_id":"4e7ddd972c5b81706ef2096d","categories":[{"id":13070,"name":"Argentinian Restaurant","short_name":"Argentinian","plural_name":"Argentinian Restaurants","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/argentinian_","suffix":".png"}}],"chains":[],"closed_bucket":"Unsure","distance":205,"geocodes":{"main":{"latitude":40.391515,"longitude":-3.696042}},"link":"/v3/places/4e7ddd972c5b81706ef2096d","location":{"admin_region":"Comunidad de Madrid","country":"ES","cross_street":"P. de la Chopera, 14","formatted_address":"P. de la Chopera, 14, 28045 Madrid Comunidad de Madrid","locality":"Madrid","postcode":"28045","region":"Comunidad de Madrid"},"name":"Cantina Matadero","related_places":{"parent":{"fsq_id":"4b6c7268f964a520d83a2ce3","categories":[{"id":12005,"name":"Cultural Center","short_name":"Cultural Center","plural_name":"Cultural Centers","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/building/default_","suffix":".png"}}],"name":"Matader

# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [24]:
import requests
from geopy.geocoders import Nominatim

venue = "coffee"
city_name = "Madrid, Spain"

In [25]:
def foursquare_places(venue, city_name):
    # Step 1: Get coordinates from city name using geopy
    geolocator = Nominatim(user_agent="geoapi")
    location = geolocator.geocode(city_name)
    
    if not location:
        print(f"Could not find coordinates for {city_name}")
        return []

    coordinates = f"{location.latitude},{location.longitude}"

In [41]:
from geopy.geocoders import Nominatim
import requests
import folium
import json

def foursquare_places(venue, city, token):
    """
    Search for places of a given type in a specified city using Foursquare API
    Returns a list of tuples containing (name, latitude, longitude)
    """
    # Step 1: Get coordinates from city name
    geolocator = Nominatim(user_agent="geoapi")
    location = geolocator.geocode(city)

    if not location:
        print(f"Could not find location for city: {city}")
        return []

    coordinates = f"{location.latitude},{location.longitude}"
    print(f"City coordinates: {coordinates}")

    # Step 2: Use Foursquare API
    url = f"https://api.foursquare.com/v3/places/search?query={venue}&ll={coordinates}&sort=DISTANCE&limit=10"

    headers = {
        "Accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Foursquare API error: {response.status_code}")
        return []

    data = response.json()
    results = data.get("results", [])
    print(f"\nFound {len(results)} {venue} locations in {city}")
    
    # Create list of places with name and coordinates
    places = []
    for place in results:
        name = place.get("name")
        lat = place.get("geocodes", {}).get("main", {}).get("latitude")
        lng = place.get("geocodes", {}).get("main", {}).get("longitude")
        places.append((name, lat, lng))
    
    # Create and save the map
    create_map(city, location, results, venue)
    
    # Display detailed information
    display_place_details(results)
    
    # Save the full JSON response
    with open("foursquare_response.json", "w") as f:
        json.dump(data, f, indent=2)
    print("\nFull response saved to foursquare_response.json")
    
    return places

def create_map(city, location, results, venue):
    """Create and save an interactive map with markers for each place"""
    map_center = [location.latitude, location.longitude]
    my_map = folium.Map(location=map_center, zoom_start=14)

    # Add a marker for the city center
    folium.Marker(
        location=map_center,
        popup=f"{city} Center",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(my_map)

    # Add markers for each venue
    for place in results:
        name = place.get("name")
        
        # Get location data
        location_data = place.get("location", {})
        address = location_data.get("formatted_address", "Address not available")
        
        # Get coordinates
        lat = place.get("geocodes", {}).get("main", {}).get("latitude")
        lng = place.get("geocodes", {}).get("main", {}).get("longitude")
        
        # Get distance
        distance = place.get("distance", "N/A")
        
        # Get hours
        hours = place.get("hours", {}).get("display", "Hours not available")
        
        # Create popup content
        popup_content = f"""
        <b>{name}</b><br>
        Address: {address}<br>
        Distance: {distance} meters<br>
        Hours: {hours}<br>
        Coordinates: {lat}, {lng}
        """
        
        # Add marker to map
        folium.Marker(
            location=[lat, lng],
            popup=folium.Popup(popup_content, max_width=300),
            icon=folium.Icon(color='green', icon='coffee', prefix='fa')
        ).add_to(my_map)

    # Step 4: Save the map to an HTML file
    map_file = f"{venue.lower()}_{city.lower()}_map.html"
    my_map.save(map_file)
    print(f"\nMap saved to {map_file}")

def display_place_details(results):
    """Display detailed information about each place in the console"""
    for i, place in enumerate(results, 1):
        print(f"\n{i}. {place.get('name')}")
        
        # Location details
        location_data = place.get("location", {})
        print(f"   Address: {location_data.get('formatted_address', 'Not available')}")
        
        # Distance
        distance = place.get("distance")
        if distance:
            print(f"   Distance: {distance} meters")
        
        # Hours
        hours = place.get("hours", {}).get("display")
        if hours:
            print(f"   Hours: {hours}")
        
        # Coordinates
        lat = place.get("geocodes", {}).get("main", {}).get("latitude")
        lng = place.get("geocodes", {}).get("main", {}).get("longitude")
        print(f"   Coordinates: {lat}, {lng}")

# 👇 Interactive usage
if __name__ == "__main__":
    venue_input = input("Enter the type of place you're looking for (e.g., Starbucks, restaurant): ")
    city_input = input("Enter the city: ")
    
    # Get Foursquare API token
    api_token = input("Enter your Foursquare API token: ")
    
    places = foursquare_places(venue_input, city_input, api_token)
    
    if places:
        print("\nTop matching places:")
        for name, lat, lng in places:
            print(f"{name} - Lat: {lat}, Lng: {lng}")
    else:
        print("No places found.")

City coordinates: 40.4167047,-3.7035825

Found 10 coffee locations in Madrid

Map saved to coffee_madrid_map.html

1. El Riojano
   Address: Calle Mayor, 10, 28013 Madrid Comunidad de Madrid
   Distance: 198 meters
   Coordinates: 40.416526, -3.705941

2. Mimi's Creperia
   Address: Calle Postas, 17, 28012 Madrid Comunidad de Madrid
   Distance: 248 meters
   Coordinates: 40.415673, -3.706148

3. El Jardín de Salvador Bachiller
   Address: Calle Montera, 37, 28013 Madrid Comunidad de Madrid
   Distance: 287 meters
   Coordinates: 40.419109, -3.702326

4. Chocolatería San Ginés
   Address: Pasadizo de San Ginés, 5 (C. Arenal), 28013 Madrid Comunidad de Madrid
   Distance: 295 meters
   Coordinates: 40.416821, -3.706735

5. La Rollerie
   Address: Calle Mayor, 24 (Coloreros 3), 28013 Madrid Comunidad de Madrid
   Distance: 299 meters
   Coordinates: 40.416473, -3.707112

6. Valdemeso
   Address: Mesonero Romanos, 8, 28013 Madrid Comunidad de Madrid
   Distance: 301 meters
   Coordinates:

# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [42]:
# your code here
from geopy.geocoders import Nominatim
import requests
import folium
import json

def foursquare_places(venue, city, token):
    """
    Search for places of a given type in a specified city using Foursquare API
    Returns a list of tuples containing (name, latitude, longitude)
    """
    # Step 1: Get coordinates from city name
    geolocator = Nominatim(user_agent="geoapi")
    location = geolocator.geocode(city)

    if not location:
        print(f"Could not find location for city: {city}")
        return []

    coordinates = f"{location.latitude},{location.longitude}"
    print(f"City coordinates: {coordinates}")

    # Step 2: Use Foursquare API
    url = f"https://api.foursquare.com/v3/places/search?query={venue}&ll={coordinates}&sort=DISTANCE&limit=10"

    headers = {
        "Accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Foursquare API error: {response.status_code}")
        return []

    data = response.json()
    results = data.get("results", [])
    print(f"\nFound {len(results)} {venue} locations in {city}")
    
    # Create list of places with name and coordinates
    places = []
    for place in results:
        name = place.get("name")
        lat = place.get("geocodes", {}).get("main", {}).get("latitude")
        lng = place.get("geocodes", {}).get("main", {}).get("longitude")
        places.append((name, lat, lng))
    
    # Create and save the map
    create_map(city, location, results, venue)
    
    # Display detailed information
    display_place_details(results)
    
    # Save the full JSON response
    with open("foursquare_response.json", "w") as f:
        json.dump(data, f, indent=2)
    print("\nFull response saved to foursquare_response.json")
    
    # Export clean data to data/coffe_shops.json
    import os
    
    # Create data directory if it doesn't exist
    os.makedirs("data", exist_ok=True)
    
    # Prepare clean data structure with relevant fields
    clean_data = []
    for place in results:
        clean_place = {
            "name": place.get("name"),
            "coordinates": {
                "latitude": place.get("geocodes", {}).get("main", {}).get("latitude"),
                "longitude": place.get("geocodes", {}).get("main", {}).get("longitude")
            },
            "address": place.get("location", {}).get("formatted_address"),
            "distance": place.get("distance"),
            "categories": [cat.get("name") for cat in place.get("categories", [])],
            "hours": place.get("hours", {}).get("display"),
            "fsq_id": place.get("fsq_id")
        }
        clean_data.append(clean_place)
    
    # Export to data/coffe_shops.json
    with open("data/coffe_shops.json", "w") as f:
        json.dump(clean_data, f, indent=2)
    print(f"Clean data exported to data/coffe_shops.json")
    
    return places

def create_map(city, location, results, venue):
    """Create and save an interactive map with markers for each place"""
    map_center = [location.latitude, location.longitude]
    my_map = folium.Map(location=map_center, zoom_start=14)

    # Add a marker for the city center
    folium.Marker(
        location=map_center,
        popup=f"{city} Center",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(my_map)

    # Add markers for each venue
    for place in results:
        name = place.get("name")
        
        # Get location data
        location_data = place.get("location", {})
        address = location_data.get("formatted_address", "Address not available")
        
        # Get coordinates
        lat = place.get("geocodes", {}).get("main", {}).get("latitude")
        lng = place.get("geocodes", {}).get("main", {}).get("longitude")
        
        # Get distance
        distance = place.get("distance", "N/A")
        
        # Get hours
        hours = place.get("hours", {}).get("display", "Hours not available")
        
        # Create popup content
        popup_content = f"""
        <b>{name}</b><br>
        Address: {address}<br>
        Distance: {distance} meters<br>
        Hours: {hours}<br>
        Coordinates: {lat}, {lng}
        """
        
        # Add marker to map
        folium.Marker(
            location=[lat, lng],
            popup=folium.Popup(popup_content, max_width=300),
            icon=folium.Icon(color='green', icon='coffee', prefix='fa')
        ).add_to(my_map)

    # Step 4: Save the map to an HTML file
    map_file = f"{venue.lower()}_{city.lower()}_map.html"
    my_map.save(map_file)
    print(f"\nMap saved to {map_file}")

def display_place_details(results):
    """Display detailed information about each place in the console"""
    for i, place in enumerate(results, 1):
        print(f"\n{i}. {place.get('name')}")
        
        # Location details
        location_data = place.get("location", {})
        print(f"   Address: {location_data.get('formatted_address', 'Not available')}")
        
        # Distance
        distance = place.get("distance")
        if distance:
            print(f"   Distance: {distance} meters")
        
        # Hours
        hours = place.get("hours", {}).get("display")
        if hours:
            print(f"   Hours: {hours}")
        
        # Coordinates
        lat = place.get("geocodes", {}).get("main", {}).get("latitude")
        lng = place.get("geocodes", {}).get("main", {}).get("longitude")
        print(f"   Coordinates: {lat}, {lng}")

# 👇 Interactive usage
if __name__ == "__main__":
    venue_input = input("Enter the type of place you're looking for (e.g., Starbucks, coffee shop): ")
    city_input = input("Enter the city: ")
    
    # Get Foursquare API token
    api_token = input("Enter your Foursquare API token: ")
    
    places = foursquare_places(venue_input, city_input, api_token)
    
    if places:
        print("\nTop matching places:")
        for name, lat, lng in places:
            print(f"{name} - Lat: {lat}, Lng: {lng}")
        print("\nData has been exported to 'data/coffe_shops.json'")
    else:
        print("No places found.")


City coordinates: 40.4167047,-3.7035825

Found 10 coffee locations in Madrid

Map saved to coffee_madrid_map.html

1. El Riojano
   Address: Calle Mayor, 10, 28013 Madrid Comunidad de Madrid
   Distance: 198 meters
   Coordinates: 40.416526, -3.705941

2. Mimi's Creperia
   Address: Calle Postas, 17, 28012 Madrid Comunidad de Madrid
   Distance: 248 meters
   Coordinates: 40.415673, -3.706148

3. El Jardín de Salvador Bachiller
   Address: Calle Montera, 37, 28013 Madrid Comunidad de Madrid
   Distance: 287 meters
   Coordinates: 40.419109, -3.702326

4. Chocolatería San Ginés
   Address: Pasadizo de San Ginés, 5 (C. Arenal), 28013 Madrid Comunidad de Madrid
   Distance: 295 meters
   Coordinates: 40.416821, -3.706735

5. La Rollerie
   Address: Calle Mayor, 24 (Coloreros 3), 28013 Madrid Comunidad de Madrid
   Distance: 299 meters
   Coordinates: 40.416473, -3.707112

6. Valdemeso
   Address: Mesonero Romanos, 8, 28013 Madrid Comunidad de Madrid
   Distance: 301 meters
   Coordinates:

# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [45]:
# your code here
from geopy.geocoders import Nominatim
import requests
import folium
import json
import os
from pymongo import MongoClient

def foursquare_places(venue, city, token):
    """
    Search for places of a given type in a specified city using Foursquare API
    Returns a list of tuples containing (name, latitude, longitude)
    """
    # Step 1: Get coordinates from city name
    geolocator = Nominatim(user_agent="geoapi")
    location = geolocator.geocode(city)

    if not location:
        print(f"Could not find location for city: {city}")
        return []

    coordinates = f"{location.latitude},{location.longitude}"
    print(f"City coordinates: {coordinates}")

    # Step 2: Use Foursquare API
    url = f"https://api.foursquare.com/v3/places/search?query={venue}&ll={coordinates}&sort=DISTANCE&limit=10"

    headers = {
        "Accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Foursquare API error: {response.status_code}")
        return []

    data = response.json()
    results = data.get("results", [])
    print(f"\nFound {len(results)} {venue} locations in {city}")
    
    # Create list of places with name and coordinates
    places = []
    for place in results:
        name = place.get("name")
        lat = place.get("geocodes", {}).get("main", {}).get("latitude")
        lng = place.get("geocodes", {}).get("main", {}).get("longitude")
        places.append((name, lat, lng))
    
    # Create and save the map
    create_map(city, location, results, venue)
    
    # Display detailed information
    display_place_details(results)
    
    # Save the full JSON response
    with open("foursquare_response.json", "w") as f:
        json.dump(data, f, indent=2)
    print("\nFull response saved to foursquare_response.json")
    
    # Export clean data to data/coffe_shops.json
    import os
    
    # Create data directory if it doesn't exist
    os.makedirs("data", exist_ok=True)
    
    # Prepare clean data structure with relevant fields
    clean_data = []
    for place in results:
        clean_place = {
            "name": place.get("name"),
            "coordinates": {
                "latitude": place.get("geocodes", {}).get("main", {}).get("latitude"),
                "longitude": place.get("geocodes", {}).get("main", {}).get("longitude")
            },
            "address": place.get("location", {}).get("formatted_address"),
            "distance": place.get("distance"),
            "categories": [cat.get("name") for cat in place.get("categories", [])],
            "hours": place.get("hours", {}).get("display"),
            "fsq_id": place.get("fsq_id")
        }
        clean_data.append(clean_place)
    
    # Export to data/coffe_shops.json
    with open("data/coffe_shops.json", "w") as f:
        json.dump(clean_data, f, indent=2)
    print(f"Clean data exported to data/coffe_shops.json")
    
    # BONUS: Insert data into MongoDB
    try:
        # Connect to MongoDB (adjust connection string if needed)
        client = MongoClient('mongodb://localhost:27017/')
        
        # Access or create the database
        db = client['venues_db']
        
        # Access or create the collection
        collection = db['ironcoffee']
        
        # Drop previous data if exists to avoid duplicates
        collection.delete_many({})
        
        # Insert the clean data into MongoDB
        if clean_data:
            collection.insert_many(clean_data)
            print(f"Successfully loaded {len(clean_data)} venues into MongoDB collection 'ironcoffee'")
        else:
            print("No data to load into MongoDB")
            
    except Exception as e:
        print(f"Error connecting to MongoDB: {e}")
        print("Please make sure MongoDB is running and pymongo is installed (pip install pymongo)")
    
    return places

def create_map(city, location, results, venue):
    """Create and save an interactive map with markers for each place"""
    map_center = [location.latitude, location.longitude]
    my_map = folium.Map(location=map_center, zoom_start=14)

    # Add a marker for the city center
    folium.Marker(
        location=map_center,
        popup=f"{city} Center",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(my_map)

    # Add markers for each venue
    for place in results:
        name = place.get("name")
        
        # Get location data
        location_data = place.get("location", {})
        address = location_data.get("formatted_address", "Address not available")
        
        # Get coordinates
        lat = place.get("geocodes", {}).get("main", {}).get("latitude")
        lng = place.get("geocodes", {}).get("main", {}).get("longitude")
        
        # Get distance
        distance = place.get("distance", "N/A")
        
        # Get hours
        hours = place.get("hours", {}).get("display", "Hours not available")
        
        # Create popup content
        popup_content = f"""
        <b>{name}</b><br>
        Address: {address}<br>
        Distance: {distance} meters<br>
        Hours: {hours}<br>
        Coordinates: {lat}, {lng}
        """
        
        # Add marker to map
        folium.Marker(
            location=[lat, lng],
            popup=folium.Popup(popup_content, max_width=300),
            icon=folium.Icon(color='green', icon='coffee', prefix='fa')
        ).add_to(my_map)

    # Step 4: Save the map to an HTML file
    map_file = f"{venue.lower()}_{city.lower()}_map.html"
    my_map.save(map_file)
    print(f"\nMap saved to {map_file}")

def display_place_details(results):
    """Display detailed information about each place in the console"""
    for i, place in enumerate(results, 1):
        print(f"\n{i}. {place.get('name')}")
        
        # Location details
        location_data = place.get("location", {})
        print(f"   Address: {location_data.get('formatted_address', 'Not available')}")
        
        # Distance
        distance = place.get("distance")
        if distance:
            print(f"   Distance: {distance} meters")
        
        # Hours
        hours = place.get("hours", {}).get("display")
        if hours:
            print(f"   Hours: {hours}")
        
        # Coordinates
        lat = place.get("geocodes", {}).get("main", {}).get("latitude")
        lng = place.get("geocodes", {}).get("main", {}).get("longitude")
        print(f"   Coordinates: {lat}, {lng}")

# 👇 Interactive usage
if __name__ == "__main__":
    venue_input = input("Enter the type of place you're looking for (e.g., Starbucks, coffee shop): ")
    city_input = input("Enter the city: ")
    
    # Get Foursquare API token
    api_token = input("Enter your Foursquare API token: ")
    
    places = foursquare_places(venue_input, city_input, api_token)
    
    if places:
        print("\nTop matching places:")
        for name, lat, lng in places:
            print(f"{name} - Lat: {lat}, Lng: {lng}")
        print("\nData has been exported to 'data/coffe_shops.json'")
    else:
        print("No places found.")


City coordinates: 40.4167047,-3.7035825

Found 10 coffee locations in Madrid

Map saved to coffee_madrid_map.html

1. El Riojano
   Address: Calle Mayor, 10, 28013 Madrid Comunidad de Madrid
   Distance: 198 meters
   Coordinates: 40.416526, -3.705941

2. Mimi's Creperia
   Address: Calle Postas, 17, 28012 Madrid Comunidad de Madrid
   Distance: 248 meters
   Coordinates: 40.415673, -3.706148

3. El Jardín de Salvador Bachiller
   Address: Calle Montera, 37, 28013 Madrid Comunidad de Madrid
   Distance: 287 meters
   Coordinates: 40.419109, -3.702326

4. Chocolatería San Ginés
   Address: Pasadizo de San Ginés, 5 (C. Arenal), 28013 Madrid Comunidad de Madrid
   Distance: 295 meters
   Coordinates: 40.416821, -3.706735

5. La Rollerie
   Address: Calle Mayor, 24 (Coloreros 3), 28013 Madrid Comunidad de Madrid
   Distance: 299 meters
   Coordinates: 40.416473, -3.707112

6. Valdemeso
   Address: Mesonero Romanos, 8, 28013 Madrid Comunidad de Madrid
   Distance: 301 meters
   Coordinates: